In [ ]:
import os
import re
import time
import pandas as pd
from pandas import json_normalize
from datetime import date
from dotenv import load_dotenv
load_dotenv()
import json
import urllib.request
import xml.etree.ElementTree as ET
from collections import Counter
import numpy as np
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# nltk.download('vader_lexicon') # Uncomment this to download the latest package when you run for the first time
import panel as pn
import holoviews as hv
import hvplot.pandas
from holoviews import opts

In [ ]:
# Set today's date:
today = date.today()

In [ ]:
# Retrieve API Key:
TOKEN = os.getenv("sec_api_key")

# Create API object:
API = "https://api.sec-api.io?token=" + TOKEN

In [ ]:
# Define API query filters: 
payload = {
  "query": { "query_string": { "query": "cik:050863" + " AND " + "filedAt:{2016-01-01 TO " + f"{today}" + "} AND " + "formType:\"10-K\"" } },
#   "from": "0",
  "size": "10",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

# Format payload to JSON:
jsondata = json.dumps(payload)
jsondataasbytes = jsondata.encode('utf-8')

# Instantiate the request: 
req = urllib.request.Request(API)

# Set the request header:
req.add_header('Content-Type', 'application/json; charset=utf-8')

# Set the request length:
req.add_header('Content-Length', len(jsondataasbytes))

# Send the request:
response = urllib.request.urlopen(req, jsondataasbytes)

# Read the response:
res_body = response.read()

# Transform the response to JSON:
filings = json.loads(res_body.decode("utf-8"))

# Print JSON:
print(filings)

In [ ]:
# Convert JSON to a DataFrame for modeling:
filings = json_normalize(filings)
filings

In [ ]:
# Create the sentiment scores list:
sentiments = []
for filing in filings['articles']:
    try:
        text = filing['content']
        date = filing['publishedAt'][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment['compound']
        pos = sentiment['pos']
        neu = sentiment['neu']
        neg = sentiment['neg']
        sentiments.append({
            'text':text,
            'date':date,
            'compound':compound,
            'positive':pos,
            'negative':neg,
            'neutral':neu 
        })
    except AttributeError:
        pass
    
# Create a DataFrame from the sentiment scores list:
sentiments_df = pd.DataFrame(sentiments)

# Reorder the columns:
columns = ['date','text','compound','positive','negative','neutral']
sentiments_df = sentiments_df[columns]